# Resolución Parcial 1 Análisis Numérico II / Álgebra Lineal Numérica

## Ejercicio 1
Sea $A \in \mathcal{R}^{n\times n}$. Si $\text{Ker}(A)$ es nulo, encontrar una base del núcleo de $B = [A \; A]$.


Si $Ker(A)$ es nulo, quiere decir que la matriz $A$ es inversible, por lo que no existe $0 \neq x \in R^n$ tal que $Ax = 0$.

Si $B = [A \; A]$, $Ker(B) = \{w \in R^{n+n} | Bw = 0 \}$
Luego si expreso $w = (x, y)$ con $x, y \in R^n$, $Bw = Ax + Ay$.
Como A tiene núcleo nulo, la única forma de que esta combinación sea nula ocurrirá cuando $y = -x$, entonces $Bw = Ax - Ax = 0$.

Por lo tanto $Ker(B) = \{ (x, -x) \in R^{n+n} \}$ y una posible base de ese conjunto, tomando a los vectores canónicos en $R^n$ $e_i$: $\{ (e_i, -e_i), i = 1, \dots, n \}$.

## Ejercicio 2
Se quiere resolver un sistema de la forma $Ax=b$, donde $A$ es simétrica, definida positiva y tiene estructura de "flecha":

$$
   A= \left(
\begin{array}{ccccc}
\star & 0 & \dots & 0 & \star \\
0& \star & \ddots & 0 & \star \\
\vdots & \ddots & \ddots & 0 & \star \\
0 & 0 & 0 & \star & \star \\
\star & \star & \star & \star & \star \\
\end{array}
\right)
$$

es decir, $A$ es diagonal excepto por su última fila y su última columna, las cuales pueden ser diferentes de cero.

- Determinar la cantidad de operaciones necesarias para obtener la descomposición de Cholesky de $A$, aprovechando su estructura.

- Implementar el algoritmo en Python y utilizarlo para resolver $Ax = b$, con

$$
A=\left(%
\begin{array}{ccccc}
3 & 0 & 0 & 0 & 1 \\
0 & 9 & 0 & 0 & 1 \\
0 & 0 & 27 & 0 & 1 \\
0 & 0 & 0 & 81 & 1 \\
1 & 1 & 1 & 1 & 243 \\
\end{array}%
\right), \hspace{1cm}
b=\left(%
\begin{array}{c}
4 \\
10 \\
28 \\
82 \\
247 \\
\end{array}%
\right)
$$

Para construir la descomposición de Cholesky de una matriz en forma de flecha, es necesario ver que, para cada iteración por columnas del algoritmo, debemos realizar sólo 2 operaciones del lado de la fila $i$ (excepto en el último paso $i = n$).

- $G_{ii} = \sqrt{A_{ii}}$ (1 flop)

- $G_{in} = A_{in} / G_{ii}$ (1 flop)

Luego, para actualizar el resto de la matriz, originalmente debemos elegir $\mathcal{J} = \{i, \dots, n \}$ y actualizar $A_{\mathcal{J} \mathcal{J}} ←A_{\mathcal{J} \mathcal{J}}- G_{i \mathcal{J}}^T G_{i \mathcal{J}}$, pero el único resultado distinto de cero (en la parte triangular superior) en el producto se dará en el elemento $G_{nn}$, por lo tanto para cada iteración (excepto en el último paso) tendremos:

- $G_{nn} = G_{nn} - G_{in}^2$ (2 flops)

Una vez que lleguemos a la iteración $n$ se realiza la última operación:

- $G_{nn} = \sqrt{G_{nn}}$ (1 flops)

Por lo tanto, la descomposición Cholesky de una matriz flecha puede realizarse en el orden de $\sum_{i=1}^n4 - 3= 4n-3$ operaciones



In [ ]:
import numpy as np

def cholesky_flecha(A):
    G = A.copy()
    n = A.shape[0]
    for idx in range(n-1):
        G[idx, idx] = np.sqrt(G[idx, idx])
        G[idx, n-1] = G[idx, n-1] / G[idx, idx]
        G[n-1, n-1] = G[n-1, n-1] - G[idx, n-1]**2
    G[n-1, n-1] = np.sqrt(G[n-1, n-1])

    return np.triu(G)

A = np.array([
    [3., 0, 0, 0, 1],
    [0, 9, 0, 0, 1],
    [0, 0, 27, 0, 1],
    [0, 0, 0, 81, 1],
    [1, 1, 1, 1, 243],
])

b = np.array([4., 10, 28, 82, 247])

G = cholesky_flecha(A)
print(G)
y = np.linalg.solve(G.T, b) # Reemplazar por soltrsup :D
x = np.linalg.solve(G, y) # Reemplazar por soltrinf :D
print(x)

[[ 1.73205081  0.          0.          0.          0.57735027]
 [ 0.          3.          0.          0.          0.33333333]
 [ 0.          0.          5.19615242  0.          0.19245009]
 [ 0.          0.          0.          9.          0.11111111]
 [ 0.          0.          0.          0.         15.5726097 ]]
[1. 1. 1. 1. 1.]


## Ejercicio 3

Sea $A \in R^{n\times n}$ una matriz simétrica y definida positiva, y sea $A = G^TG$ su descomposición de Cholesky. Mostrar que existe la descomposción $LU$ de $A$ y que es única. Dar los factores de $L$ y $U$ en términos de $G$.

Si $A$ es simétrica y definida positiva, entonces todos sus menores principales tienen determinante distinto de cero, por lo tanto su descomposición LU existe y es única si pedimos que todos los elementos de L sean unos.

Si $A = G^TG$, proponemos la matriz $D = diag(G)$. Al ser G la matriz de descomposición de Cholesky de $A$, todos los elementos de la diagonal son no nulos, entonces existe $D^{-1}$, matriz diagonal con $D_{ii}^{-1} = \frac{1}{D_{ii}}$ para $i = 1, \dots, n$.

Ahora,

$ A = G^TG = G^TIG = G^TD^{-1}DG = (G^TD^{-1}) (DG)$

donde $G^T D^{-1}$ es un producto de una matriz triangular inferior y una matriz diagonal, entonces es una matriz triangular inferior. Además $G^T D^{-1}_{ii} = \frac{G_{ii}}{D_{ii}} = \frac{D_{ii}}{D_{ii}} = 1$ para $i = 1, \dots, n$, entonces todos los elementos de su diagonal son unos.

Luego, $DG$ es un producto de una matriz diagonal por una triangular superior, por lo que es triangular superior.

Por lo tanto $L = G^T D^{-1}$ y $U = DG$ es la descomposición LU de $A$.

## Ejercicio 4

Considerar la siguiente matriz simétrica:
$$ A =
    \left[
    \begin{array}{ccc}
    a & a & a\\
    a& b & b\\
    a & b & c\\
    \end{array}%
    \right].
$$
    
- Encontrar las condiciones que deben cumplir $a, b$ y $c$ para evitar pivots nulos al realizar la descomposición $LU$.
- Implementar una función en Python cuyas entradas sean los números $a, b$ y $c$. La función debe verificar las condiciones y devolver los factores $L$ y $U$. Caso contrario, imprimir el mensaje "no se satisfacen las condiciones".  

Supongamos que queremos realizar la descomposición LU sin pivoteo de la matriz $A$, luego de realizar las operaciones asociadas a la primera columna obtendremos la siguiente matriz:

$$ A_1 =
    \left[
    \begin{array}{ccc}
    a & a & a\\
    0 & b-a & b-a\\
    0 & b-a & c-a\\
    \end{array}%
    \right].
$$

Al finalizar las operaciones relacionadas a la segunda columna, nos quedaremos con la siguiente descomposición:

$$ A_2 =
    \left[
    \begin{array}{ccc}
    a & a & a\\
    0 & b-a & b-a\\
    0 & 0 & c-a - (b-a)\\
    \end{array}%
    \right].
$$

Para que la descomposición pueda ser realizada sin realizar ningún pivoteo, los elementos de la diagonal deben ser no nulos, por lo tanto las condiciones que deben cumplir $a, b$ y $c$ son:

- $a \neq 0$
- $b-a \neq 0$
- $c-a-(b-a) = c-b \neq 0$

La matriz $L$ asociada a esta descomposición tiene la siguiente forma:

$$
L =
    \left[
    \begin{array}{ccc}
    1 & 0 & 0\\
    -1 & 1 & 0\\
    -1 & -1 & 1\\
    \end{array}%
    \right].
$$

In [ ]:
from scipy import linalg
def descomposicion_abc(a, b, c):
    L, U = None, None
    if a == 0 or b == a or c == b:
        print("no se satisfacen las condiciones")
    else:
        L = np.array([[1., 0, 0], [1, 1, 0], [1, 1, 1]])
        U = np.array([[a, a, a], [0, b-a, b-a], [0, 0, c-b]])
    return L, U

### TEST Y COMPARACION CON SCIPY LINALG

L, U = descomposicion_abc(1, 2, 3)
print(L, U)

A = np.array([
    [1., 1, 1],
    [1, 2, 2],
    [1, 2, 3],
])
L, U, P = linalg.lu(A)
print(L, U)

[[1. 0. 0.]
 [1. 1. 0.]
 [1. 1. 1.]] [[1 1 1]
 [0 1 1]
 [0 0 1]]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]] [[1. 0. 0.]
 [1. 1. 0.]
 [1. 1. 1.]]


## Ejercicio 5

Probar que para toda $A\in R^{n\times n}$ no singular:
- Si $||.||$ es una norma inducida, entonces $\| A \| \| A^{-1} \| \geq 1$,
- $\kappa_F(A) \geq \sqrt{n}$.

- Si $||.||$ es una norma inducida entonces es submultiplicativa y la norma de la matriz identidad es $||I|| = 1$ (pues $sup_{x \ne 0} \frac{\|Ix\|}{\|x\|} = sup_{x \ne 0} \frac{\|x\|}{\|x\|} = 1$)  , por lo tanto

$$
1 = \|I\| = \|A A^{-1}\| \le \|A\| \|A^{-1}\|
$$

- La norma Frobenius es submultiplicativa y la norma Frobenius de $I$ es
$\|I\|_F = \sqrt{tr(I^TI)} = \sqrt{tr(I)} = \sqrt{n}$. Si aplico lo mismo de arriba se obtiene que

$$
\sqrt{n} = \|I\|_F = \|A A^{-1}\|_F \le \|A\|_F \|A^{-1}\|_F = \kappa_F(A)
$$

